# Семинар 1 - Первичный анализ данных, визуализация, etc. 

__Источник данных:__ [https://opendata.socrata.com/](https://opendata.socrata.com/Government/Airplane-Crashes-and-Fatalities-Since-1908/q2te-8cvq)

__Dataset:__ Airplane Crashes and Fatalities Since 1908

Содержит полную историю авиакатастроф по всему миру, с 1908 года-по настоящее время.
> `../data/Airplane_Crashes_and_Fatalities_Since_1908.csv`

## 0. Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np

import scipy.stats as st

%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(palette='deep', style='darkgrid', rc={"figure.figsize": (15, 8)})


import warnings
warnings.simplefilter('ignore')

## 1. Чтение данных и просмотр
Воспользуйтесь функцией .read_csv()

In [ ]:
df = pd.read_csv("../data/Airplane_Crashes_and_Fatalities_Since_1908.csv")


#Пока не смотрим на этот код: 
df['Time'] = df['Time'].str.replace('c:','')

- __Date__: Дата инцидента
- __Time__: Время инцидента
- __Location__: Локация инцидента
- __Operator__: Авиакомпания
- __Flight #__
- __Route__
- __Type__: Тип разбившегося самолета 
- __Registration__:
- __cn/In__
- __Aborad__: Сколько людей было на борту самолета
- __Fatalities__: Число погибших (из тех, кто был на борту)
- __Ground:__ Число погибших на земле (из тех, кто не был на борту)
- __Summary__: Краткое описание инцидента



In [ ]:
# Посмотрим на наш датасет, а так же применим поочередно классы .head(), .tail(), .columns
# Ваш Код здесь: 
df.head()

По умолчанию Pandas выводит всего 20 столбцов и 60 строк, поэтому если ваш датафрейм больше, воспользуйтесь функцией set_option:
> pd.set_option('display.max_columns', 100)  
> pd.set_option('display.max_rows', 100)

In [ ]:
# .shape позволяет выводить размер датасета. Можно обращаться к выводу как к масиву с помощью []
print(df.shape[0], 'строк')
print(df.shape[1], 'столбцов')

In [ ]:
#сформируем таблицу с типами данных с помощью класса .dtypes:
df_info = pd.DataFrame(df.dtypes,columns=['Type'])
df_info

In [ ]:
# Посмотрим уникальные значения Авиаоператоров с помощью.unique() и их кол-во .nunique() 
df['Operator'].nunique()

In [ ]:
#Бывает удобнее воспользоваться классом .info()
df.info()

## 2. Работа с датой
Как мы видим колнка Date иммет тип "object", хотя pandas умеет работать с типом данных "datetime64"

In [ ]:
#смените тип данных для колонки "Date" с помощью .to_datetime(), обратите внимание на параметры (dayfirst, yearfirst)
#
# Ошибки можно проигнорировать параметром errors = 'coerce'

df['Date'] = pd.to_datetime(df['Date'])
df['Time'] = pd.to_datetime(df['Time'], errors = 'coerce')
df.dtypes

Что нам дало изменение типа? В частности теперь мы можем доставать значение года (.dt.year), месяца (.dt.month), дня (.dt.day) и даже дня недели(.dt.dayofweek, Понедельник – 0, Воскресенье – 6)

Кстати, чтобы добавить колонку, достаточно просто присвоить значения к df['Название новой колонки']

In [ ]:
#Достаньте из 'Date' - Год, месяц, день недели. 

df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day
df['day of week'] = df['Date'].dt.dayofweek # Monday = 0, Sunday = 6,

df['Hours'] = df['Time'].dt.hour
df.head(3)

## 3. Обращение к данным, фильтрация и другое

In [ ]:
#Посмотрите сколько инцидентов было в 2002 году: 
df[df['year'] == 2002].shape[0]

In [ ]:
#Посмотрите сколько инцидентов было c кол-вом жертв (суммарно на земле и на борту) больше 200: 
df[df['Fatalities'] + df['Ground'] > 200].shape[0]

In [ ]:
# Кроме того мы можем производить любые операции со столбцами складывать/умножать/делить/логарифимировать etc.   
df['all_victims'] = df['Fatalities'] + df['Ground']

In [ ]:
# Посомтрите сколько инцидентов было c кол-вом жертв (суммарно на земле и на борту) больше 200 в 2002 году: 
df[(df['year'] == 2002) & (df['all_victims'] > 200)].shape[0]

In [ ]:
# Где произошел инцидент? Какой был самолет? Сколько было жертв? 
print('В 2002 году в ', df[(df['year'] == 2002) & (df['all_victims'] > 200)]['Location'].values[0],
      'разбился', df[(df['year'] == 2002) & (df['all_victims'] > 200)]['Type'].values[0], '. '
      'Общее кол-во жертв', df[(df['year'] == 2002) & (df['all_victims'] > 200)]['all_victims'].values[0]
     )

In [ ]:
# Самостоятельно прочитать про .loc / .iloc

# 4. Работа с текстом и пропусками, объединение таблиц

In [ ]:
# Посмотрим на столбец 'Location'
df['Location'].head()

In [ ]:
# Как видим локация расписана через запятую, попробуем разбить название с помощью .str.split
loc = df['Location'].str.split(',', expand=True)
loc.head()

#кстати в .str есть много всего полезного, например используемый ранее .replace()

Видно, что у нас появились пропуски, однако чем правее тем более укрупненно можно посмотреть локацию. 
Пропуски можно удалить с помощью .drop(), однако нам более интересно заполнить их с помощью fillna()
Не забывайте, что есть параметр inplace

In [ ]:
loc[3] = loc[3].fillna(loc[2])
loc[3] = loc[3].fillna(loc[1])
loc[3] = loc[3].fillna(loc[0])
loc[3].nunique()

In [ ]:
# объединим таблицы с помощью pd.merge()
df_full = pd.merge(df, loc, left_index=True, right_index=True)
df_full.head(2)

In [ ]:
# Удалите колонки '0', '1', '2' с помощью .drop() 
# переименуйте колонку '3' в  'Country' (по крайней мере мы надеемся, что там страна)
df_full = df_full.drop([0,1,2], axis=1)
df_full.columns.values[19] = 'Country'
df_full.head(3)

In [ ]:
# Сделаем признаки: Day_period и Holiday с помощью .apply(lambda x: ...)

def get_day_time(hour):
    return {
          0 <= hour < 6:   'Ночь',
          6 <= hour < 11:  'Утро',
         11 <= hour < 19:  'День',
         19 <= hour < 23:  'Вечер',
         23 <= hour < 25:  'Ночь'
    }[True]
        
df_full['Day_period'] = df_full['Hours'].fillna(0).apply(lambda x: get_day_time(x))
df_full['Holiday'] = df_full['day of week'].fillna(0).apply(lambda x: 1 if x >= 5 else 0) 

# .map на самостоятельное изучение

## 5. Сводные таблицы, группировка и агрегация данных 

In [ ]:
# C помощью .groupby посчитайте сумму жертв по годам. Затем отсортируйте их по убыванию и сделайте топ-10 
df_full.groupby('year').sum().sort_values('all_victims', ascending=False).head(10)

In [ ]:
# Нарисуйте график all_victims, Aboard, Fatalities для каждого года
df.groupby('year').sum()[['all_victims', 'Ground', 'Fatalities']].plot(rot=45) #kind='bar'
plt.title('Количество жерт по годам')
plt.show()

In [ ]:
# Посчитайте кол-во Инцидентов и жертв по годам. На этот раз используйте .pivot_table()

crashes_by_year = pd.pivot_table(df_full, index='year', values=['Aboard','Fatalities','Ground','all_victims'], aggfunc='sum')
crashes_by_year['count'] =  pd.pivot_table(df_full, index='year', values=['all_victims'], aggfunc='count').values.T[0]
crashes_by_year.tail()

In [ ]:
# Сколько жертв на инцидент? 
crashes_by_year['victims_on_incident'] = crashes_by_year['all_victims']/crashes_by_year['count']

In [ ]:
# существует ли корреляция между всеми праметрами? 
crashes_by_year.corr()

In [ ]:
#Отрисуйте heatmap корреляции для более хорошего визульного представления - используйте sns.heatmap() 

sns.heatmap(crashes_by_year.corr(), 
         vmin=-1, vmax=1,annot=True, cmap='RdBu',)
plt.show()

# Бонус! Давайте порисуем еще! 

In [ ]:
_, axes = plt.subplots(1, 2, sharey=True, figsize=(16,6))

sns.boxplot(x="Day_period", y="Fatalities", data=df_full[df_full['Fatalities'] < 100],  ax=axes[0]);
sns.violinplot(x="Day_period", y="Fatalities", data=df_full[df_full['Fatalities'] < 100], ax=axes[1],
              hue='Holiday');

### Разберём части Boxplot ###

__Черта__ – Медиана  
__Коробка__ – Интерквартильный размах (IQR или разница 25% (Q1) и 75% (Q3) перцентили)   
__Усы__ – Это интервал $\bigl[ Q_1 - 1.5 \times IQR$, $Q_3 + 1.5 \times IQR \bigr]$  
__Точки__ – Выбросы

<img src="../pic/Boxplot.svg" width="500">
Подообнее на [Wikipedia]('https://en.wikipedia.org/wiki/Box_plot')

In [ ]:
fig = plt.figure(figsize=(22, 5))
ax1 = fig.add_subplot(121)
sns.distplot(df_full['Fatalities'].dropna(), fit=st.norm, kde=True,ax=ax1, bins=50)
ax1.set_title('Плотность распределения переменной')
ax2 = fig.add_subplot(122)
prob = st.probplot(df_full['Fatalities'].dropna(), dist=st.norm, plot=ax2)
ax2.set_title('Probabilyty plot')
plt.show()

### Ссылки:
- Хороший [обзор основных функций Pandas](https://alexanderdyakonov.files.wordpress.com/2015/04/ama2015_pandas.pdf) от Александра Дьяконова (МГУ)
- Открытый курс машинного обучения. [Тема 1. Первичный анализ данных с Pandas](https://habr.com/company/ods/blog/322626/)  
- Подробная [Документация Pandas](http://pandas.pydata.org/pandas-docs/stable/api.html)

### Литература: 
- Heydt Michael. Learning Pandas 